In [ ]:
import json 
import boto3

def lambda_handler(event,context):
    bucket_name = ''
    session = boto3.Session(aws_access_key_id='',aws_secret_access_key='')
    s3 = session.client('s3')
    kinises = session.client('firehose',region_name='us-east-1')
    
    keys = s3.list_objects_v2(Bucket=bucket_name)['Contents']
    for key in keys:
        file = s3.get_object(Bucket = bucket_name, Key =key['Key'])
    
        pragraphs = str(file['Body'].read().decode("utf-8"))[:-1]
        pragraphs = pragraphs.split("\n")
        
        
        for pragraph in pragraphs:
            pragraph = json.loads(pragraph)
            
            Text = pragraph['text']
            #pass to comperhend and get the response
            comperhand = session.client('comprehend',region_name='us-east-1')
            response = comperhand.detect_sentiment(Text = Text, LanguageCode = 'en')
            # add it to the record
            pragraph['Sentiment'] = response['Sentiment']
            pragraph['Sentiment']
            response = kinises.put_record(
                       DeliveryStreamName='',
                       Record={
                            'Data': json.dumps(pragraph)+'\n'
                        }
            )
        s3.delete_object(Bucket=bucket_name, Key=key['Key'])